In [761]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import os


In [762]:
pd.set_option('display.max_columns', None)
df=pd.read_csv("../Data/Dataframelimpa_sem_latlong.csv")


## Preparando a base para o modelo

In [763]:
df_order=df["order_id"]
df=df[["product_category_name","delivery_time","product_weight_g","product_length_cm","product_height_cm","product_width_cm",
       "price","freight_value"]]



In [764]:
df.dropna(inplace=True)

In [765]:
df["product_category_name"]=pd.Categorical(df["product_category_name"])
df["product_category_name"]=df["product_category_name"].cat.codes




In [766]:
def outliers(coluna):
    q1=np.percentile(df[coluna],25)
    q3=np.percentile(df[coluna],75)

    iqr=q3-q1
    out_sup=q3+iqr*1.5
    out_inf=q1-iqr*1.5
    return(out_inf,out_sup)

inf,sup=outliers("freight_value")
#df[df["freight_value"]>sup].sort_values(by="freight_value",ascending=False).head(50)
df.loc[df["freight_value"]>300,"freight_value"]=300


In [767]:
df[df["freight_value"]>sup].sort_values(by="freight_value",ascending=False).head(50)

,product_category_name,delivery_time,product_weight_g,product_length_cm,product_height_cm,product_width_cm,price,freight_value
63902,40,16.30,225.0,18.0,4.0,12.0,13.98,300.00
570,10,8.30,950.0,45.0,15.0,35.0,69.90,300.00
63431,31,10.18,1200.0,54.0,8.0,11.0,198.00,300.00
37733,40,4.89,200.0,22.0,8.0,15.0,19.90,300.00
14324,33,6.83,80.0,17.0,9.0,14.0,10.25,300.00
11009,41,7.31,2700.0,25.0,15.0,25.0,69.00,300.00
47634,41,4.96,1600.0,25.0,25.0,25.0,64.99,300.00
37440,6,18.43,1050.0,30.0,5.0,20.0,124.99,300.00
50712,31,13.92,1200.0,69.0,11.0,11.0,119.00,300.00
9376,6,3.29,2800.0,36.0,3.0,24.0,139.00,300.00


## Aplicando o Modelo

In [768]:
y=df["delivery_time"]
x=df.drop(columns=["delivery_time"])

In [769]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.35,random_state=10)
#x_train.describe()

In [770]:
reg=GradientBoostingRegressor()
reg.fit(x_train,y_train)


GradientBoostingRegressor()

In [771]:

y_pred=reg.predict(x_test)

mse=mean_squared_error(y_test,y_pred)

In [772]:
print(mse)

24.821937347168223


In [773]:
comparação=pd.DataFrame()
comparação["delivery_time"]=y_test
comparação["Modelo ML"]=y_pred
comparação

,delivery_time,Modelo ML
63492,18.63,14.859363
32660,9.87,12.680614
58211,14.78,11.901218
37664,22.82,13.261177
35107,13.38,8.683675
...,...,...
63404,6.13,11.858348
41916,7.47,9.998034
28515,14.01,10.410535
56212,13.16,11.187924


## Agora para a base do Kaggle

In [774]:
kaggle=pd.read_csv("../Data/Kaggle_enriquecido.csv")

In [775]:
categorias_a_agrupar = {
    'construcao_ferramentas_ferramentas': 'construcao_ferramentas',
    'construcao_ferramentas_construcao': 'construcao_ferramentas',
    'construcao_ferramentas_jardim': 'construcao_ferramentas',
    'construcao_ferramentas_iluminacao': 'construcao_ferramentas',
    'construcao_ferramentas_seguranca': 'construcao_ferramentas',
    'ferramentas_jardim': 'construcao_ferramentas',
    'moveis_sala': 'moveis',
    'moveis_quarto': 'moveis',
    'moveis_colchao_e_estofado': 'moveis',
    'moveis_cozinha_area_de_servico_jantar_e_jardim': 'moveis',
    'moveis_decoracao': 'moveis',
    'moveis_escritorio': 'moveis',
    'pc_gamer': 'pcs',
    'artes_e_artesanato':'artes',
    'telefonia_fixa':'telefonia',
    'alimentos': 'alimentos_bebidas',
    'bebidas': 'alimentos_bebidas',
    'cds_dvds_musicais': 'cds_dvds',
    'dvds_blu_ray': 'cds_dvds',
    'portateis_casa_forno_e_cafe': 'eletroportateis',
    'casa_conforto_2': 'casa_conforto',
    'eletrodomesticos_2': 'eletrodomesticos',
    'malas_acessorios':'fashion',
    'fashion_bolsas_e_acessorios':'fashion',
    'fashion_calcados':'fashion',
    'fashion_underwear_e_moda_praia':'fashion',
    'fashion_roupa_masculina':'fashion',
    'fashion_esporte':'fashion' ,
    'fashion_roupa_feminina':'fashion',
    'fashion_roupa_infanto_juvenil':'fashion',
    'eletronicos':'informatica_acessorios',
    'tablets_impressao_imagem':'informatica_acessorios',
    'la_cuisine':'utilidades_domesticas',
    'fraldas_higiene': 'bebes'
    
}

kaggle['product_category_name'].replace(categorias_a_agrupar, inplace=True)


In [776]:
kaggle["product_category_name"]=pd.Categorical(kaggle["product_category_name"])
kaggle["product_category_name"]=kaggle["product_category_name"].cat.codes


In [777]:
x_kaggle=kaggle[["product_category_name","product_weight_g","product_length_cm","product_height_cm","product_width_cm",
       "price","freight_value"]]

In [778]:
x_kaggle["product_category_name"].unique()

array([31, 10, 12,  6,  1, 15, 17,  8,  9, 40, 25, 19, 38,  7, 41, 18, 35,
        5, 33, 24, 37,  0, 21,  4, 36, 16, 29, 26, 22, -1, 20, 27, 28, 11,
       14, 39, 34,  2, 30, 13, 32,  3, 23], dtype=int8)

In [779]:
x_kaggle["freight_value"].fillna(x_kaggle["freight_value"].median(),inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_27212\208030238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_kaggle["freight_value"].fillna(x_kaggle["freight_value"].median(),inplace=True)


In [780]:
x_kaggle["price"].fillna(x_kaggle["price"].median(), inplace=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_27212\3247522267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_kaggle["price"].fillna(x_kaggle["price"].median(), inplace=True)


In [781]:
x_kaggle.isna().sum()

product_category_name    0
product_weight_g         0
product_length_cm        0
product_height_cm        0
product_width_cm         0
price                    0
freight_value            0
dtype: int64

In [782]:
y_kaggle=reg.predict(x_kaggle)
len(y_kaggle)

28152

In [783]:
original=pd.read_csv("../Data/csv_Kaggle.csv")

In [784]:
original["order_metric_cycle_time"]=y_kaggle
display(original)

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_metric_cycle_time
0,cee3292f46ede6ea1dfabfcb200fcf47,0f8feee05f830b2be5955cf674a7d8f1,6/21/2018 15:00,6/21/2018 17:18,11.898619
1,50dca53ca33b739bef09e7933e8b380e,d0580aed7f303a707d93b00b84c591a7,7/14/2018 09:47,7/14/2018 10:04,9.965385
2,8087ec71e393d4dc6fc48041fe63cd51,b8dfb418e6cad762b91febf827d4b2e5,5/14/2018 10:03,5/14/2018 10:15,6.625668
3,e6b6557ce111de79b31cc857f20ba212,90c8fe0d707ea62be3fcb5abd7bc784f,5/22/2018 18:41,5/22/2018 18:57,11.979022
4,0b09c5e4c2512f627190ac55a78c35a3,6ba83000cf8c24413916a7a72fcb5173,4/30/2018 19:31,5/3/2018 04:12,13.731538
...,...,...,...,...,...
28147,81cedc4d2132eb6f512354166e51b187,a8f6bb35d78515a1762eb626d52b6352,7/30/2018 21:02,7/30/2018 21:10,9.667107
28148,95811675a89b66659996da5285a021db,5885969ba920706beb8d5381a2ba981c,8/9/2018 11:07,8/9/2018 15:25,12.965257
28149,25bd2ad9d282f59e4e6a048ef9bf214f,be34bf56044c5614fc66d97251c0adc6,3/12/2018 16:20,3/12/2018 17:09,17.227104
28150,9f9ac312604229686c04260955495c70,3357c4c3ecf82709236158cb2ac0ce9b,5/24/2018 11:38,5/24/2018 11:53,6.703501


In [785]:
original.drop(columns=["customer_id","order_purchase_timestamp","order_approved_at"],inplace=True)
original

,order_id,order_metric_cycle_time
0,cee3292f46ede6ea1dfabfcb200fcf47,11.898619
1,50dca53ca33b739bef09e7933e8b380e,9.965385
2,8087ec71e393d4dc6fc48041fe63cd51,6.625668
3,e6b6557ce111de79b31cc857f20ba212,11.979022
4,0b09c5e4c2512f627190ac55a78c35a3,13.731538
...,...,...
28147,81cedc4d2132eb6f512354166e51b187,9.667107
28148,95811675a89b66659996da5285a021db,12.965257
28149,25bd2ad9d282f59e4e6a048ef9bf214f,17.227104
28150,9f9ac312604229686c04260955495c70,6.703501


## Processo de tunagem do modelo

In [786]:
from sklearn.model_selection import GridSearchCV

In [787]:
reg=GradientBoostingRegressor()

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(reg, param_grid, cv=3,
scoring='neg_mean_squared_error')

grid_search.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 4, 5], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300],
                         'subsample': [0.8, 0.9, 1.0]},
             scoring='neg_mean_squared_error')

In [788]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print((-1)*mean_score, params)

27.072987619912976 {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.8}
27.080930218208238 {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.9}
27.085698197941735 {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 1.0}
25.93272856107977 {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200, 'subsample': 0.8}
25.942511128171734 {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200, 'subsample': 0.9}
25.95604510507574 {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200, 'subsample': 1.0}
25.569566874637804 {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300, 'subs

In [789]:
""" caminho_arquivo = os.path.join("../Data", 'resultado_previsoes_kaggle_2.csv')
original.to_csv(caminho_arquivo,index=False) """

' caminho_arquivo = os.path.join("../Data", \'resultado_previsoes_kaggle_2.csv\')\noriginal.to_csv(caminho_arquivo,index=False) '